In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
attrition_df = pd.read_csv('./resources/HR-Employee-Attrition.csv')
attrition_cat = attrition_df.dtypes[attrition_df.dtypes == 'object'].index.tolist()

##Encoding the categorical values

#create encoder instance
enc = OneHotEncoder(sparse=False)

#fit and transform the model using the categorical values
encode_df = pd.DataFrame(enc.fit_transform(attrition_df[attrition_cat]))

#add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(attrition_cat)
attrition_df = attrition_df.merge(encode_df, left_index=True, right_index=True).drop(attrition_cat, axis=1)
#Split our preprocessed data for training and testing

y = attrition_df['Attrition_Yes'].values
X = attrition_df.drop(['Attrition_Yes', 'Attrition_No'],1).values

#Split the data

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=78)
#Scaling
scaler = StandardScaler()

#Fit scaler
X_scaler = scaler.fit(X_train)

#Transform dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
#Define the model - deep neural net
number_input = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

#first hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim = number_input, activation = 'relu'))

#second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation = 'relu'))

#output layer

nn.add(tf.keras.layers.Dense(units=1, activation = 'sigmoid'))

nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 448       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 499
Trainable params: 499
Non-trainable params: 0
_________________________________________________________________


In [3]:
#Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

In [4]:
#define checkpoint path and filenames
os.makedirs('checkpoints/', exist_ok=True)
checkpoint_path='checkpoints/weights.{epoch:02d}.hdf5'

In [6]:
#compile the model

nn.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')

#create a callback that saves the model's weight every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only = True,
    save_freq = 1000
    )

#fit the model
nn.fit(X_train_scaled,y_train, epochs=100, callbacks=[cp_callback])

#evaluate the model
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f'Loss: {model_loss}, Acc:{model_accuracy}')

Epoch 1/100
35/35 [==============================] - 0s 618us/step - loss: 0.6342 - accuracy: 0.7831
Epoch 2/100
35/35 [==============================] - 0s 677us/step - loss: 0.5640 - accuracy: 0.8153
Epoch 3/100
35/35 [==============================] - 0s 647us/step - loss: 0.5252 - accuracy: 0.8045
Epoch 4/100
35/35 [==============================] - 0s 882us/step - loss: 0.4757 - accuracy: 0.8466
Epoch 5/100
35/35 [==============================] - 0s 647us/step - loss: 0.4953 - accuracy: 0.7942
Epoch 6/100
35/35 [==============================] - 0s 647us/step - loss: 0.4432 - accuracy: 0.8335
Epoch 7/100
35/35 [==============================] - 0s 706us/step - loss: 0.4285 - accuracy: 0.8237
Epoch 8/100
35/35 [==============================] - 0s 677us/step - loss: 0.4137 - accuracy: 0.8234
Epoch 9/100
35/35 [==============================] - 0s 676us/step - loss: 0.4162 - accuracy: 0.8150
Epoch 10/100
35/35 [==============================] - 0s 765us/step - loss: 0.3746 - accura

35/35 [==============================] - 0s 676us/step - loss: 0.2044 - accuracy: 0.9309
Epoch 79/100
35/35 [==============================] - 0s 676us/step - loss: 0.1860 - accuracy: 0.9312
Epoch 80/100
35/35 [==============================] - 0s 618us/step - loss: 0.1873 - accuracy: 0.9322
Epoch 81/100
35/35 [==============================] - 0s 618us/step - loss: 0.1928 - accuracy: 0.9277
Epoch 82/100
35/35 [==============================] - 0s 618us/step - loss: 0.1796 - accuracy: 0.9320
Epoch 83/100
35/35 [==============================] - 0s 588us/step - loss: 0.1839 - accuracy: 0.9382
Epoch 84/100
35/35 [==============================] - 0s 618us/step - loss: 0.1756 - accuracy: 0.9431
Epoch 85/100
35/35 [==============================] - 0s 647us/step - loss: 0.1822 - accuracy: 0.9334
Epoch 86/100
 1/35 [..............................] - ETA: 0s - loss: 0.1322 - accuracy: 0.9688
Epoch 00086: saving model to checkpoints\weights.86.hdf5
35/35 [==============================] - 0s 

In [8]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_new.load_weights("checkpoints/weights.86.hdf5")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.4138 - accuracy: 0.8668
Loss: 0.41378507018089294, Accuracy: 0.866847813129425


In [9]:
# Export our model to HDF5 file
nn_new.save("./saves/trained_attrition.h5")

In [10]:
nn_imported = tf.keras.models.load_model("./saves/trained_attrition.h5")

In [11]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.4138 - accuracy: 0.8668
Loss: 0.41378507018089294, Accuracy: 0.866847813129425
